<h1><center>Лабораторная работа 3. Методы поиска подстроки в строке.</center></h1>

<h2><center>Выполнил студент группы БВТ1901 Атаев Реджеп</center></h2>

<h3><b>Задание №1</b></h3>

Реализовать методы поиска подстроки в строке. Добавить возможность
ввода строки и подстроки с клавиатуры. Предусмотреть возможность
существования пробела. Реализовать возможность выбора опции
чувствительности или нечувствительности к регистру. Оценить время работы
каждого алгоритма поиска и сравнить его со временем работы стандартной
функции поиска, используемой в выбранном языке программирования.

1. [Кнута-Морриса-Пратта](https://en.wikipedia.org/wiki/Knuth%E2%80%93Morris%E2%80%93Pratt_algorithm)

2. [Упрощенный Бойера-Мура](https://en.wikipedia.org/wiki/Boyer%E2%80%93Moore%E2%80%93Horspool_algorithm)</li>

In [1]:
def prefixkmp(word):
    pi = [0] * len(word)
    length = 0
    iterator = 1
    while iterator < len(word):
        if word[iterator] == word[length]:  # Ai == Aj
            pi[iterator] = length + 1
            length += 1
            iterator += 1
        else:
            if length == 0:                 # Ai != Aj and j == 0
                pi[iterator] = 0
                iterator += 1
            else:                           # Ai != !j and j != 0
                length = pi[length - 1]

    return pi

In [11]:
def kmp(text, word): #Knuth-Morris-Pratt Algoritm
    coords = [] # first and last index of word we search
    length = 0

    p = prefixkmp(word)

    for i in range(len(text)):
        while length > 0 and text[i] != word[length]:
            length = p[length - 1]
        if text[i] == word[length]:
            length += 1
        if length == len(word):
            coords.append((i - len(word) + 1, i + 1))
            length = p[length - 1]

    return(f"KMP Word begins:ends at indexes {coords}")

In [12]:
def prefixbm(word):
    alphabet = [len(word)]*256
    for i in range(len(word)-1):
        alphabet[ord(word[i])] = len(word) - 1 - i
    return alphabet

In [13]:
def bm(string, word): #Boyer–Moore–Horspool algorithm
    coords = []
    T = prefixbm(word)
    skip = 0
    while len(string) - skip >= len(word):
        if string[skip:skip + len(word)] == word:
            coords.append((skip, skip + len(word)))
        skip += T[ord(string[skip + len(word) - 1])]
    return (f"BM Word begins:ends at indexes {coords}")

In [14]:
print(bm("Hoola-Hoola Girls like Hooligans", "Hooligan"))
print(kmp("Hoola-Hoola Girls like Hooligans", "Hooligan"))

BM Word begins:ends at indexes [(23, 31)]
KMP Word begins:ends at indexes [(23, 31)]


<h3><b>Задание №2</b></h3>

Игра в 15, пятнашки, такен — популярная головоломка, придуманная
в 1878 году Ноем Чепмэном. Она представляет собой набор
одинаковых квадратных костяшек с нанесёнными числами, заключённых в
квадратную коробку. Длина стороны коробки в четыре раза больше длины
стороны костяшек для набора из 15 элементов, соответственно в коробке
остаётся незаполненным одно квадратное поле. Цель игры — перемещая
костяшки по коробке, добиться упорядочивания их по номерам, желательно
сделав как можно меньше перемещений.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/f/ff/15-puzzle_magical.svg/220px-15-puzzle_magical.svg.png)
![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/91/15-puzzle.svg/220px-15-puzzle.svg.png)

<ul>
    <li><b>Задача</b>: написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.</li>
    <li><b>Входные данные</b>: массив чисел, представляющий собой расстановку в
порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.</li>
    <li><b>Выходные данные</b>: если решения нет, то функция должна вернуть
пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента.</li>
    <li><b>Например</b>:, для начального расположения элементов [1, 2, 3, 4, 5, 6, 7, 8,
13, 9, 11, 12, 10, 14, 15, 0] одним из возможных решений будет [15, 14, 10, 13,
9, 10, 14, 15] (последовательность шагов здесь: двигаем 15, двигаем 14,
двигаем 10, …, двигаем 15).</li>
</ul>

In [1]:
from queue import PriorityQueue
from random import shuffle

N = 4
SEARCH_TYPE = "greedy"  # implemented: greedy

def moves(position):
    blank = position.index(N * N - 1)
    i, j = divmod(blank, N)
    offsets = []
    if i > 0:
        offsets.append(-N)  # Down
    if i < N - 1:
        offsets.append(N)  # Up
    if j > 0:
        offsets.append(-1)  # Right
    if j < N - 1:
        offsets.append(1)  # Left
    for offset in offsets:
        swap = blank + offset
        yield tuple(
            position[swap] if x == blank else position[blank] if x == swap else position[x] for x in range(N * N))


def loss(position):
    return sum(abs(i // N - position[i] // N) + abs(i % N - position[i] % N) for i in range(N * N - 1))


def parity(permutation):
    # assert set(permutation) == set(range(N*N))
    # return sum(x<y and px>py for (x, px) in enumerate(permutation) for (y, py) in enumerate(permutation))%2
    seen, cycles = set(), 0
    for i in permutation:
        if i not in seen:
            cycles += 1
            while i not in seen:
                seen.add(i)
                i = permutation[i]
    return (cycles + len(permutation)) % 2


class Position:  # For PriorityQueue, to make "<" do the right thing.
    def __init__(self, position, start_distance):
        self.position = position
        self.loss = loss(position)
        self.start_distance = start_distance

    def __lt__(self, other):
        if SEARCH_TYPE == "greedy":
            return self.loss < other.loss
        else:
            raise NotImplementedError

    def __str__(self):
        return '\n'.join(
            (N * '{:3}').format(*[(i + 1) % (N * N) for i in self.position[i:]]) for i in range(0, N * N, N))


start = list(range(N * N - 1))
while 1:
    shuffle(start)
    if parity(start) == 0: break
start += [N * N - 1]
start = tuple(start)
print(start)
p = Position(start, 0)
candidates = PriorityQueue()
candidates.put(p)
visited = {p}  # Tuples rather than lists so they go into a set.
came_from = {p.position: None}

while p.position != tuple(range(N * N)):
    p = candidates.get()
    for k in moves(p.position):
        if k not in visited:
            candidates.put(Position(k, p.start_distance + 1))
            came_from[k] = p
            visited.add(k)

while p.position != start:
    print(p, "\n")
    p = came_from[p.position]

(13, 12, 14, 7, 4, 0, 8, 6, 5, 10, 2, 11, 1, 3, 9, 15)
  1  2  3  4
  5  6  7  8
  9 10 11 12
 13 14 15  0 

  1  2  3  4
  5  6  7  8
  9 10 11 12
 13 14  0 15 

  1  2  3  4
  5  6  7  8
  9 10 11 12
 13  0 14 15 

  1  2  3  4
  5  6  7  8
  9  0 11 12
 13 10 14 15 

  1  2  3  4
  5  6  7  8
  9 11  0 12
 13 10 14 15 

  1  2  3  4
  5  6  7  8
  9 11 14 12
 13 10  0 15 

  1  2  3  4
  5  6  7  8
  9 11 14 12
 13 10 15  0 

  1  2  3  4
  5  6  7  8
  9 11 14  0
 13 10 15 12 

  1  2  3  4
  5  6  7  8
  9 11  0 14
 13 10 15 12 

  1  2  3  4
  5  6  7  8
  9  0 11 14
 13 10 15 12 

  1  2  3  4
  5  6  7  8
  9 10 11 14
 13  0 15 12 

  1  2  3  4
  5  6  7  8
  9 10 11 14
 13 15  0 12 

  1  2  3  4
  5  6  7  8
  9 10 11 14
 13 15 12  0 

  1  2  3  4
  5  6  7  8
  9 10 11  0
 13 15 12 14 

  1  2  3  4
  5  6  7  8
  9 10  0 11
 13 15 12 14 

  1  2  3  4
  5  6  7  8
  9 10 12 11
 13 15  0 14 

  1  2  3  4
  5  6  7  8
  9 10 12 11
 13 15 14  0 

  1  2  3  4
  5  6  7  8
 